# Walmart Kaggle Project

## Authors:  Shijie Wang and Matthew Odom

In [2]:
%ls

README.md              sample_submission.csv  test.csv.zip
Walmart_Script.ipynb   test.csv               train.csv


### Install packages necessary for computation and load the datasets

In [108]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import random

# load and prepare the training dataset for modeling
train = pd.read_csv("train.csv")
train = train.dropna()
train = train.ix[:, (train.columns != 'FinelineNumber') & (train.columns != 'Upc')]

# aggregate ScanCount by DepartmentDescription for each VisitNumber
data_1 = train.pivot_table(index = 'VisitNumber', columns = 'DepartmentDescription', values = 'ScanCount', 
                           aggfunc = np.sum, fill_value=0)
data_1['VisitNumber'] = data_1.index

# get unique Trip Type, VisitNumber, and Weekday
temp = train.drop(train.columns[[3,4]], axis=1)
temp = temp.drop_duplicates()

train = pd.merge(temp, data_1, on = 'VisitNumber')
train = pd.get_dummies(train)



# load and prepare the testing dataset for modeling
test = pd.read_csv("test.csv")
test = test.dropna()
test = test.ix[:, (test.columns != 'FinelineNumber') & (test.columns != 'Upc')]

# aggregate ScanCount by DepartmentDescription for each VisitNumber
data_1 = test.pivot_table(index = 'VisitNumber', columns = 'DepartmentDescription', values = 'ScanCount', 
                           aggfunc = np.sum, fill_value=0)
data_1['VisitNumber'] = data_1.index

# get unique Trip Type, VisitNumber, and Weekday
temp = test.drop(test.columns[[2,3]], axis=1)
temp = temp.drop_duplicates()

test = pd.merge(temp, data_1, on = 'VisitNumber')
test = pd.get_dummies(test)

X_test = test.ix[:,test.columns != 'VisitNumber']
X_test['Intercept'] = 1

# a subset for algorithm build
#data = train[train["TripType"] <= 10]
#data = data.sample(50000)
data = train

### Create a loop for calculating each TripType

In [110]:
# create a loop for each TripType
# stack the output of the test into single submission
TripTypes = data.TripType.unique()
submission = pd.DataFrame()

for i in TripTypes:
    print "Working on TripType " + str(i)
    
    # reset dataset for next iteration
    sample = data
    sample = sample.dropna()
    
    # convert independent variable into binary
    sample.TripType = np.where(sample.TripType != i, 0, 1) 
    sample['Intercept'] = 1
    
    # set up model
    X = sample.ix[:,(sample.columns != 'TripType') & (sample.columns != 'VisitNumber') & (sample.columns != 'HEALTH AND BEAUTY AIDS')]
    y = sample.ix[:,sample.columns == 'TripType']
    
    model = sm.OLS(y,X)
    results = model.fit()
    
    # predict test dataset values
    yhat = pd.DataFrame(results.predict(X_test)) # this should be results.predict(test)
    yhat.columns = ["TripType_%s" % i]
    
    # append columns to submission
    submission = submission.join(yhat, how = "outer")
    
print "All done"

Working on TripType 999
Working on TripType 30
Working on TripType 26
Working on TripType 8
Working on TripType 35
Working on TripType 41
Working on TripType 21
Working on TripType 6
Working on TripType 42
Working on TripType 7
Working on TripType 9
Working on TripType 39
Working on TripType 25
Working on TripType 38
Working on TripType 15
Working on TripType 36
Working on TripType 20
Working on TripType 37
Working on TripType 32
Working on TripType 40
Working on TripType 5
Working on TripType 3
Working on TripType 4
Working on TripType 24
Working on TripType 33
Working on TripType 43
Working on TripType 31
Working on TripType 27
Working on TripType 34
Working on TripType 18
Working on TripType 29
Working on TripType 44
Working on TripType 19
Working on TripType 23
Working on TripType 22
Working on TripType 28
Working on TripType 14
Working on TripType 12
All done


In [ ]:
# clean up submission dataset for Kaggle submission
submission = submission.sort_index(axis=1)
submission = submission.apply(lambda row: np.where(row == max(row), 1, 0), axis = 1)

test['VisitNumber'].j



In [107]:
pd.DataFrame(test['VisitNumber']).join(submission)

,VisitNumber,TripType_12,TripType_14,TripType_15,TripType_18,TripType_19,TripType_20,TripType_21,TripType_22,TripType_23,...,TripType_41,TripType_42,TripType_43,TripType_44,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [151]:
# sample of summary
print results.summary()

                            OLS Regression Results                            
Dep. Variable:               TripType   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     378.1
Date:                Sat, 21 Nov 2015   Prob (F-statistic):               0.00
Time:                        22:47:28   Log-Likelihood:                -5161.5
No. Observations:               48456   AIC:                         1.048e+04
Df Residuals:                   48379   BIC:                         1.115e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------